# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

In [68]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Саня\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
def simple_reader():
    with open(r"Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt", 'r') as f:
        text = " ".join(f.read().replace("\n", ' ').split(' '))

    sentences = nltk.sent_tokenize(text)
    return sentences

answer = simple_reader()

In [70]:
print(list(map(lambda x: len(x), answer))[:20])
print("Олейников Адександр")

[287, 133, 41, 60, 90, 48, 69, 140, 130, 61, 38, 36, 32, 17, 55, 50, 52, 56, 96, 30]
Олейников Адександр


2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

In [71]:
print(list(filter(lambda x: len(x) <= 10, answer))[:10])
print("Олейников Александр")

['— ton).]', '—\xa0Плюнуть?', '«Как!', '—\xa0Почему?', '—\xa0Как!', 'Чудо!', 'А?', 'Как зачем?', '—\xa0Как?', '—\xa0Кого?']
Олейников Александр


3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

In [72]:
import dask.bag as db
b = db.from_sequence(answer, npartitions=10)
result = (b.map(len).fold(lambda x, y: x+y, lambda x, y: x+y))
print(result.compute()/len(answer))
print("Олейников Александр")

72.89840944073885
Олейников Александр


4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

In [73]:
import json
with open(r"addres-book.json", "r") as read_file:
    data = json.load(read_file)

In [74]:
def read_phones(dictionary):
    phones = []
    for p in dictionary.get('phones', []):
        phones.append(p.get('phone', ''))
    return phones

In [75]:
data = db.from_sequence(data)
mobile_phones = data.map(read_phones).flatten().filter(lambda x: '+' in x).count()
print(f"Мобильных: {mobile_phones.compute()}")
work_phones = data.map(read_phones).flatten().filter(lambda x: '+' not in x).count()
print(f"Рабочих: {work_phones.compute()}")
print("Олейников Александр")

Мобильных: 7
Рабочих: 6
Олейников Александр


## Лабораторная работа 12

In [1]:
import dask.bag as db
import json
import re
import pandas as pd

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [2]:
data = db.read_text(r"reviews_full/reviews_*.json").map(json.loads)
print("Олейников Алесандр")
data.take(5)

Олейников Алесандр


({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [32]:
data2 = db.read_text(r"reviews_full/reviews_*.json", include_path=True)
data2.take(5)

(('{"user_id": 452355, "recipe_id": 292657, "date": "2016-05-08", "review": "WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing."}\n',
  'U:/Python/PyCharm/pythonProject/lr12/reviews_full/reviews_0.json'),
 ('{"user_id": 329304, "recipe_id": 433404, "date": "2006-06-14", "review": "This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten"}\n',
  'U:/Python/PyCharm/pythonProject/lr12/reviews_full/reviews_0.json'),
 ('{"user_id": 227932, "recipe_id": 2008187, "date": "1985-11-19", "review": "Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( whi

In [4]:
def find_and_add(note):
    data = json.loads(note[0])
    data["rating"] = int(re.findall("\d+.json", note[1])[0].split('.')[0])
    return data

In [5]:
print("Олейников Александр")
data_with_rating = data2.map(lambda x : find_and_add(x))
data_with_rating.take(5)

Олейников Александр


({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will

3. Посчитайте количество отзывов в исходном датасете.

In [6]:
print("Олейников Александр")
data_with_rating.count().compute()

Олейников Александр


9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [9]:
data_filter = data_with_rating.filter(lambda x: x['date'].split('-')[0] in ('2014', '2015'))
data_filter.take(5)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese and baked, uncovered, for the 25 minutes.  Served with pico de gallo, s

In [10]:
print("Олейников Александр")
data_filter.count().compute()

Олейников Александр


735274

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [11]:
def preprocessing(node):
    node["review"] = re.sub(r'[^A-Za-z ]+', '', node["review"].lower().strip())
    return node

print("Олейников Александр")
data_preprocessing = data_filter.map(lambda x: preprocessing(x))
data_preprocessing.take(5)

Олейников Александр


({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'simple and easy way to enjoy a slice of pizza any time  welltoasted bread is the key  really toast it  i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor  i used sourdough bread  medium cheddar cheese  fast  fun  great idea  made for  hits tag game',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'delish  i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans  i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic  added the olives under the cheese and baked uncovered for the  minutes  served with pico de gallo sour cream and avocado chunks  fantastic  tha

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [28]:
print("Олейников Александр")
data_preprocessing.count().compute()

Олейников Александр


735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка

In [30]:
# pluck - выбор rating ключа из всех словарей
dict(data_preprocessing.pluck("rating").frequencies())

{0: 42472, 1: 9246, 2: 9380, 3: 26532, 4: 119413, 5: 528231}

Олейников Александр

8. Найдите среднее значение `rating` в выборке

In [31]:
data_preprocessing.pluck('rating').mean().compute()

4.388036296673077

Олейников Александр

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [41]:
key_func = lambda x: x['rating']
reduce_func = lambda last, current: max(last, current, key=lambda item:len(item['review']))

data_preprocessing.foldby(key_func, reduce_func).map(lambda x: {x[0]: len(x[1]['review'])}).compute()

[{0: 6548}, {1: 2868}, {2: 2834}, {3: 3174}, {4: 6548}, {5: 5343}]